# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 2:26PM,238687,22,616638,"NBTY Global, Inc.",Released
1,Jun 2 2022 2:25PM,238686,22,616639,"NBTY Global, Inc.",Released
2,Jun 2 2022 2:22PM,238685,22,616632,"NBTY Global, Inc.",Released
3,Jun 2 2022 2:20PM,238684,21,616595,"NBTY Global, Inc.",Released
4,Jun 2 2022 2:18PM,238682,20,8086764,"ACI Healthcare USA, Inc.",Released
5,Jun 2 2022 2:18PM,238682,20,ACI-8091181,"ACI Healthcare USA, Inc.",Released
6,Jun 2 2022 2:18PM,238682,20,8091182,"ACI Healthcare USA, Inc.",Released
7,Jun 2 2022 2:18PM,238682,20,ACI-8091745,"ACI Healthcare USA, Inc.",Released
8,Jun 2 2022 2:18PM,238682,20,ACI-8091749,"ACI Healthcare USA, Inc.",Released
9,Jun 2 2022 2:18PM,238682,20,8091764,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,238683,Released,1
52,238684,Released,1
53,238685,Released,1
54,238686,Released,1
55,238687,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238683,NaN,NaN,1.0
238684,NaN,NaN,1.0
238685,NaN,NaN,1.0
238686,NaN,NaN,1.0
238687,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,5.0,5.0,4.0
238597,0.0,21.0,1.0
238598,0.0,0.0,2.0
238613,1.0,2.0,0.0
238616,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,5,5,4
238597,0,21,1
238598,0,0,2
238613,1,2,0
238616,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,5,5,4
1,238597,0,21,1
2,238598,0,0,2
3,238613,1,2,0
4,238616,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,5,5,4
1,238597,,21,1
2,238598,,,2
3,238613,1,2,
4,238616,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc."
1,Jun 2 2022 2:25PM,238686,22,"NBTY Global, Inc."
2,Jun 2 2022 2:22PM,238685,22,"NBTY Global, Inc."
3,Jun 2 2022 2:20PM,238684,21,"NBTY Global, Inc."
4,Jun 2 2022 2:18PM,238682,20,"ACI Healthcare USA, Inc."
26,Jun 2 2022 2:18PM,238683,20,Alumier Labs Inc.
27,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC"
30,Jun 2 2022 1:51PM,238678,10,Emerginnova
37,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation
82,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc.",,,1
1,Jun 2 2022 2:25PM,238686,22,"NBTY Global, Inc.",,,1
2,Jun 2 2022 2:22PM,238685,22,"NBTY Global, Inc.",,,1
3,Jun 2 2022 2:20PM,238684,21,"NBTY Global, Inc.",,,1
4,Jun 2 2022 2:18PM,238682,20,"ACI Healthcare USA, Inc.",,,22
5,Jun 2 2022 2:18PM,238683,20,Alumier Labs Inc.,,,1
6,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC",,,3
7,Jun 2 2022 1:51PM,238678,10,Emerginnova,,,7
8,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation,,,45
9,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc.",1,,
1,Jun 2 2022 2:25PM,238686,22,"NBTY Global, Inc.",1,,
2,Jun 2 2022 2:22PM,238685,22,"NBTY Global, Inc.",1,,
3,Jun 2 2022 2:20PM,238684,21,"NBTY Global, Inc.",1,,
4,Jun 2 2022 2:18PM,238682,20,"ACI Healthcare USA, Inc.",22,,
5,Jun 2 2022 2:18PM,238683,20,Alumier Labs Inc.,1,,
6,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC",3,,
7,Jun 2 2022 1:51PM,238678,10,Emerginnova,7,,
8,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation,45,,
9,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc.",1,,
1,Jun 2 2022 2:25PM,238686,22,"NBTY Global, Inc.",1,,
2,Jun 2 2022 2:22PM,238685,22,"NBTY Global, Inc.",1,,
3,Jun 2 2022 2:20PM,238684,21,"NBTY Global, Inc.",1,,
4,Jun 2 2022 2:18PM,238682,20,"ACI Healthcare USA, Inc.",22,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 2 2022 2:25PM,238686,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 2 2022 2:22PM,238685,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 2 2022 2:20PM,238684,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jun 2 2022 2:18PM,238682,20,"ACI Healthcare USA, Inc.",22.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 2 2022 2:25PM,238686,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 2 2022 2:22PM,238685,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 2 2022 2:20PM,238684,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jun 2 2022 2:18PM,238682,20,"ACI Healthcare USA, Inc.",22.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3579872,120.0,36.0,5.0
12,954561,8.0,4.0,1.0
15,1431714,13.0,36.0,20.0
16,715969,3.0,0.0,0.0
19,954626,13.0,6.0,0.0
20,1431990,36.0,9.0,0.0
21,238684,1.0,0.0,0.0
22,716058,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3579872,120.0,36.0,5.0
1,12,954561,8.0,4.0,1.0
2,15,1431714,13.0,36.0,20.0
3,16,715969,3.0,0.0,0.0
4,19,954626,13.0,6.0,0.0
5,20,1431990,36.0,9.0,0.0
6,21,238684,1.0,0.0,0.0
7,22,716058,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,120.0,36.0,5.0
1,12,8.0,4.0,1.0
2,15,13.0,36.0,20.0
3,16,3.0,0.0,0.0
4,19,13.0,6.0,0.0
5,20,36.0,9.0,0.0
6,21,1.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,120.0
1,12,Released,8.0
2,15,Released,13.0
3,16,Released,3.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,5.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0
Executing,36.0,4.0,36.0,0.0,6.0,9.0,0.0,0.0
Released,120.0,8.0,13.0,3.0,13.0,36.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,5.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0
1,Executing,36.0,4.0,36.0,0.0,6.0,9.0,0.0,0.0
2,Released,120.0,8.0,13.0,3.0,13.0,36.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,5.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0
1,Executing,36.0,4.0,36.0,0.0,6.0,9.0,0.0,0.0
2,Released,120.0,8.0,13.0,3.0,13.0,36.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()